# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БФИ2001 Заморский Пётр Владимирович
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [1]:
# Общие структуры для описания графов
class Node:
    def __init__(self, index):
        self.index = index
        self.connections = []

    def __repr__(self):
        return f"Node {self.index}"

    def connect_to(self, node, cost, double = False):
        a = Connection(node, cost)
        self.connections.append(a)
        if double: 
            b = Connection(self, cost)
            node.connections.append(b)
            return [a, b]
        return [a]

    def restore_connection(self, connection):
        self.connections.append(connection)

    def remove_connections_to(self, node):
        i = 0
        r = []
        while i < len(self.connections):
            if self.connections[i].node == node:
                r.append(self.connections.pop(i))
                continue
            i+=1
        return r

    def remove_all_in_connections(self, nodes):
        r = []
        for n in nodes:
            r.extend([(n, c) for c in n.remove_connections_to(self)])
        return r

    def get_connection(self, to_node):
        for c in self.connections:
            if c.node == to_node:
                return c
        return None

class Connection:
    def __init__(self, to_node, cost):
        self.node = to_node
        self.cost = cost

class PathfindingResult:
    def __init__(self, total_cost, path):
        self.total_cost = total_cost
        self.path = path
    
    def __repr__(self):
        return f"Path with cost '{self.total_cost}' - {self.path}"

In [2]:
# Алгоритм Флойда-Уоршелла
class FloydWarshall:
    class NodeState:
        def __init__(self, node):
            self.node = node
            self.to = dict()

    def calc_path(from_node, to_node, nodes):
        states = [FloydWarshall.NodeState(n) for n in nodes]
        def get_state_for(node):
            for s in states:
                if s.node == node:
                    return s
            return None
        for i in states:
            for c in i.node.connections:
                i.to[get_state_for(c.node)] = (c.cost, None)
        for k in states:
            for i in states:
                if k == i: continue
                for j in states:
                    if j == k or j == i: continue
                    a = i.to.get(k, (99999, None))[0]
                    b = k.to.get(j, (99999, None))[0]
                    s = a + b
                    if i.to.get(j, (99999, None))[0] > s:
                        i.to[j] = (s, k)
        from_state = get_state_for(from_node)
        to_state = get_state_for(to_node)
        path = []
        current = to_state
        while current is not None:
            path.append(current.node)
            current = from_state.to.get(current, (99999, None))[1]
        path.append(from_state.node)
        path.reverse()
        return PathfindingResult(from_state.to[to_state][0], path)

In [3]:
# Алгоритм Дейкстры
class Dijkstra:
    class NodeState:
        def __init__(self, node, cost):
            self.node = node
            self.cost = cost
            self.best_from = None

    def calc_path(from_node, to_node, nodes):
        if from_node is None:
            print("'From' must be valid Node object")
            return
        if to_node is None:
            print("'To' must be valid Node object")
            return

        open = [Dijkstra.NodeState(from_node, 0)]
        close = []

        def get_min_open():
            min = open[0]
            for n in open:
                if n.cost < min.cost:
                    min = n
            return min
        def is_node_closed(node): return get_closed_state(node) is not None 
        def get_closed_state(node): return get_state_from_node(close, node)
        def get_opened_state(node): return get_state_from_node(open, node)
        def get_state_from_node(states, node):
            for n in states:
                if n.node == node:
                    return n
            return None
        def calc_costs_for_connections(origin_state):
            for connection in origin_state.node.connections:
                if is_node_closed(connection.node): continue
                node_state = get_opened_state(connection.node)
                if node_state is None:
                    node_state = Dijkstra.NodeState(connection.node, 9999999)
                    open.append(node_state)
                current_cost = origin_state.cost + connection.cost
                if current_cost < node_state.cost:
                    node_state.cost = current_cost
                    node_state.best_from = origin_state
        def create_result():
            destination_state = get_closed_state(to_node)
            path = []
            current = destination_state
            while current is not None:
                path.append(current.node)
                current = current.best_from
            path.reverse()
            return PathfindingResult(destination_state.cost, path)

        while len(open) > 0:
            current = get_min_open()
            calc_costs_for_connections(current)
            open.remove(current)
            close.append(current)

        return create_result()

In [4]:
# Алгоритм Беллмана-Форда
class BellmanFord:
    def calc_path(from_node, to_node, nodes, raw = False):
        d = [[(99999, -1) for _ in range(len(nodes))] for _ in range(len(nodes))]
        d[nodes.index(from_node)][0] = (0, -1)

        for i in range(1, len(nodes)):
            for index_from in range(len(nodes)):
                for c in nodes[index_from].connections:
                    index_to = nodes.index(c.node)
                    s = d[index_from][i - 1][0] + c.cost
                    if d[index_to][i][0] > s:
                        d[index_to][i] = (s, index_from)

        if raw: return d

        min_path_len = 0
        min_path_cost = 99999
        to_node_index = nodes.index(to_node)
        for i in range(len(d[to_node_index])):
            if min_path_cost > d[to_node_index][i][0]:
                min_path_cost = d[to_node_index][i][0]
                min_path_len = i
        path = []
        current_node = to_node_index
        for i in range(min_path_len, 0, -1):
            path.append(nodes[current_node])
            current_node = d[current_node][i][1]
        path.append(from_node)
        path.reverse()
        return PathfindingResult(min_path_cost, path)

In [5]:
# Алгоритм Джонсона
class Johnson:
    def calc_path(from_node, to_node, nodes):
        temp_node = Node(-1)
        nodes.append(temp_node)
        for n in nodes:
            temp_node.connect_to(n, 0)
        
        bell_dist = BellmanFord.calc_path(temp_node, None, nodes, raw=True)
        
        for i in range(len(nodes)):
            for c in nodes[i].connections:
                c.cost += bell_dist[i][-1][0] - bell_dist[nodes.index(c.node)][-1][0]
        
        nodes.pop()

        result = Dijkstra.calc_path(from_node, to_node, nodes)
        result.total_cost += bell_dist[nodes.index(from_node)][-1][0] - bell_dist[nodes.index(to_node)][-1][0]

        for i in range(len(nodes)):
            for c in nodes[i].connections:
                c.cost += -bell_dist[i][-1][0] + bell_dist[nodes.index(c.node)][-1][0]
        
        return result


In [6]:
# Алгоритм Левита
from collections import deque

class Levit:
    class NodeState:
        def __init__(self, node, cost, status):
            self.node = node
            self.cost = cost
            self.previous = None
            self.status = status
    
    def calc_path(from_node, to_node, nodes):
        states = {}
        for n in nodes:
            states[n] = Levit.NodeState(n, 
                0 if n == from_node else 99999,
                1 if n == from_node else 2)
        
        q = deque([states[from_node]])

        while q:
            current = q.pop()
            current.status = 0
            for c in current.node.connections:
                to_state = states[c.node]
                if to_state.cost > current.cost + c.cost:
                    to_state.cost = current.cost + c.cost
                    if to_state.status == 2:
                        q.appendleft(to_state)
                    elif to_state.status == 0:
                        q.append(to_state)
                    to_state.previous = current
                    to_state.status = 1

        destination_state = states[to_node]
        path = []
        current = destination_state
        while current is not None:
            path.append(current.node)
            current = current.previous
        path.reverse()
        return PathfindingResult(destination_state.cost, path)

In [7]:
# Алгоритм Йена
class Yen:
    def calc_path(from_node, to_node, nodes, count = 1):
        initial_path = Dijkstra.calc_path(from_node, to_node, nodes)
        paths = [initial_path]
        B = []

        for k in range(1, count + 1):
            for i in range(len(paths[k - 1].path) - 2):
                removed_connections = []
                temp_nodes = nodes.copy()
                spur_node = paths[k - 1].path[i]
                root_path = paths[k - 1].path[0:i]

                for p in paths:
                    if root_path == p.path[0:i]:
                        removed = p.path[i].remove_connections_to(p.path[i + 1])
                        removed_connections.extend([(p.path[i], c) for c in removed])
                
                for n in root_path:
                    if n == spur_node: continue
                    removed = n.remove_all_in_connections(temp_nodes)
                    removed_connections.extend(removed)
                    temp_nodes.remove(n)
                
                spur_path = Dijkstra.calc_path(spur_node, to_node, temp_nodes)
                total_path = root_path + spur_path.path

                cost = 0
                for i in range(len(total_path) - 1):
                    c = total_path[i].get_connection(total_path[i + 1])
                    cost += c.cost

                if (cost, total_path) not in B:
                    B.append((cost, total_path))
                
                for n, c in removed_connections:
                    n.restore_connection(c)

            if len(B) == 0: break
            shortest_path_cost = 999999
            shortest_path_index = 0
            for i in range(len(B)):
                if shortest_path_cost > B[i][0]:
                    shortest_path_cost = B[i][0]
                    shortest_path_index = i
            
            paths.append(PathfindingResult(B[shortest_path_index][0], B[shortest_path_index][1]))
            B.pop(shortest_path_index)
        
        return paths


In [8]:
# Демонстрация работы с графом, задаваемым напрямую кодом
# (пример из википедии - https://ru.wikipedia.org/wiki/Алгоритм_Дейкстры)
nodes = [Node(i + 1) for i in range(6)]

nodes[0].connect_to(nodes[1], 7, True)
nodes[0].connect_to(nodes[2], 9, True)
nodes[0].connect_to(nodes[5], 14, True)

nodes[1].connect_to(nodes[2], 10, True)
nodes[1].connect_to(nodes[3], 15, True)

nodes[2].connect_to(nodes[3], 11, True)
nodes[2].connect_to(nodes[5], 2, True)

nodes[3].connect_to(nodes[4], 6, True)

nodes[4].connect_to(nodes[5], 9, True)

def calc_path_with_all_methods(from_node, to_node, nodes):
    methods = [
        ("Floyd-Warshall", FloydWarshall.calc_path),
        ("Dijkstra", Dijkstra.calc_path),
        ("Bellman-Ford", BellmanFord.calc_path),
        ("Johnson", Johnson.calc_path),
        ("Levit", Levit.calc_path),
        ("Yen", Yen.calc_path)
    ]
    for m in methods:
        print(f"{m[0]}:\n   {m[1](from_node, to_node, nodes)}")

calc_path_with_all_methods(nodes[0], nodes[4], nodes)

Floyd-Warshall:
   Path with cost '20' - [Node 1, Node 3, Node 6, Node 5]
Dijkstra:
   Path with cost '20' - [Node 1, Node 3, Node 6, Node 5]
Bellman-Ford:
   Path with cost '20' - [Node 1, Node 3, Node 6, Node 5]
Johnson:
   Path with cost '20' - [Node 1, Node 3, Node 6, Node 5]
Levit:
   Path with cost '20' - [Node 1, Node 3, Node 6, Node 5]
Yen:
   [Path with cost '20' - [Node 1, Node 3, Node 6, Node 5], Path with cost '23' - [Node 1, Node 6, Node 5]]


In [9]:
# Чтение матрицы смежности из файла
def read_map_from_file(file):
    map = [[int(n.strip()) for n in line.split(',')] for line in file.readlines()]
    nodes = [Node(i + 1) for i in range(len(map))]
    for row in range(len(map)):
        for col in range(len(map)):
            if map[row][col] > 0:
                nodes[row].connect_to(nodes[col], map[row][col])
    return nodes

# Демонстрация:
with open("Lab6_data/map.csv") as file:
    nodes = read_map_from_file(file)

calc_path_with_all_methods(nodes[0], nodes[-1], nodes)

Floyd-Warshall:
   Path with cost '11' - [Node 1, Node 3, Node 6]
Dijkstra:
   Path with cost '11' - [Node 1, Node 3, Node 6]
Bellman-Ford:
   Path with cost '11' - [Node 1, Node 3, Node 6]
Johnson:
   Path with cost '11' - [Node 1, Node 3, Node 6]
Levit:
   Path with cost '11' - [Node 1, Node 3, Node 6]
Yen:
   [Path with cost '11' - [Node 1, Node 3, Node 6], Path with cost '14' - [Node 1, Node 6]]


In [10]:
# GUI (описание в md-блоке ниже)
import tkinter as tk
from tkinter import ttk
import math

node_size = 25
nodes = []
next_node_index = 0
start_node = None
def set_start_node(node):
    global start_node
    if start_node is not None:
        start_node.widget.itemconfig(start_node.widget.oval_id, outline='black')
    start_node = node
    if start_node is not None:
        start_node.widget.itemconfig(start_node.widget.oval_id, outline='red')

def mouse_pressed(event):
    if event.widget != main_canvas: return
    create_node(event.x, event.y)

def mouse_with_control(event):
    if hasattr(event.widget, "node"):
        node = event.widget.node
        if start_node is None:
            set_start_node(node)
            return
        if start_node == node:
            set_start_node(None)
            return
        
        new_connections = start_node.connect_to(node, cost.get(), is_both_sided.get())
        draw_nodes_connection(start_node, node, cost.get(), is_both_sided.get(), new_connections)
        set_start_node(None)

def mouse_pressed_on_node(event):
    if event.num == 3:
        remove_node(event.widget.node)

def create_node(x, y):
    global next_node_index
    index = next_node_index
    next_node_index += 1
    widget = draw_node(x, y, str(index + 1))
    node = Node(index)
    nodes.append(node)
    node.widget = widget
    node.coords = (x, y)
    widget.node = node
    return node

def draw_node(x, y, label):
    c = tk.Canvas(main_canvas, width=node_size, height=node_size)
    c.oval_id = c.create_oval(2,2,node_size-2,node_size-2)
    c.create_text(node_size/2, node_size/2, text=label)
    c.place(x=x-node_size/2, y=y-node_size/2)
    c.bind("<Button>", mouse_pressed_on_node)
    c.lines = []
    return c

def remove_node(node):
    for c in node.connections:
        main_canvas.delete(c.g[0])
        main_canvas.delete(c.g[1])
        main_canvas.delete(c.g[2])
    node.widget.destroy()
    node.remove_all_in_connections(nodes)
    nodes.remove(node)

def draw_nodes_connection(a, b, cost, double_sided, connections):
    arrow = tk.BOTH if double_sided else tk.LAST
    a_coords = a.coords
    b_coords = b.coords
    dir = vector_normalized(vector_add(vector_multiply(a_coords, -1), b_coords))
    line_start = vector_add(a_coords, vector_multiply(dir, node_size * 0.75))
    line_end = vector_add(b_coords, vector_multiply(dir, node_size * -0.75))
    
    line = main_canvas.create_line(line_start[0], line_start[1], line_end[0], line_end[1], arrow=arrow)
    label = main_canvas.create_text((a.coords[0] + b.coords[0]) / 2, (a.coords[1] + b.coords[1]) / 2, text=cost)
    label_bg = main_canvas.create_rectangle(main_canvas.bbox(label), fill="white")
    main_canvas.tag_lower(label_bg, label)
    for c in connections:
        c.g = (line, label, label_bg)

def vector_multiply(v, s):
    return (v[0] * s, v[1] * s)

def vector_add(a, b):
    return (a[0] + b[0], a[1] + b[1])

def vector_magnitude(v):
    return math.sqrt(v[0] ** 2 + v[1] ** 2)

def vector_normalized(v):
    return vector_multiply(v, 1 / vector_magnitude(v))

def find_node_with_index(index):
    for n in nodes:
        if n.index == index:
            return n
    return None

def on_calculate_path_button():
    from_node = find_node_with_index(path_from_index.get() - 1)
    to_node = find_node_with_index(path_to_index.get() - 1)
    if from_node is None:
        print(f"Can not find path: node {path_from_index.get()} not found")
        return
    if to_node is None:
        print(f"Can not find path: node {path_to_index.get()} not found")
        return
    result = methods[selected_method.get()](from_node, to_node, nodes)
    set_start_node(None)
    display_path(result.path)
    path_cost_label['text'] = f"Path cost: {result.total_cost}"

def display_path(path):
    for n in nodes:
        n.widget.itemconfig(n.widget.oval_id, outline='black')
        for c in n.connections:
            main_canvas.itemconfig(c.g[0], fill='black', width=1)
    for i in range(len(path) - 1):
        cur_node = path[i]
        next_node = path[i + 1]
        cur_node.widget.itemconfig(cur_node.widget.oval_id, outline='green')
        connection = cur_node.get_connection(next_node)
        main_canvas.itemconfig(connection.g[0], fill='green', width=3)
    last_node = path[-1]
    last_node.widget.itemconfig(last_node.widget.oval_id, outline='green')

methods = {
    "Floyd-Warshall": FloydWarshall.calc_path,
    "Dijkstra": Dijkstra.calc_path,
    "Bellman-Ford": BellmanFord.calc_path,
    "Johnson": Johnson.calc_path,
    "Levit": Levit.calc_path,
}

root = tk.Tk()
root.title("Поиск пути в графе")
window_width = 700
window_height = 500
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
center_x = int(screen_width/2 - window_width / 2)
center_y = int(screen_height/2 - window_height / 2)
root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y}')
root.bind("<Control-Button-1>", mouse_with_control)

main_canvas = tk.Canvas(root)
main_canvas.grid(column=0)
main_canvas.bind("<Button>", mouse_pressed)
main_canvas.pack(expand=True, fill='both', side='left')

ttk.Separator(root, orient='vertical').pack(fill='both', side='left')

frame = ttk.Frame(root)
frame.columnconfigure(0, weight=1)
frame.columnconfigure(0, weight=1)
frame.pack(expand=True, fill='both')

ttk.Label(frame, text="Connect nodes with CTRL+Click:").grid(column=0, columnspan=2, row=0)

ttk.Label(frame, text="Cost:").grid(column=0, row=1)
cost = tk.IntVar(root, 1)
cost_entry = ttk.Entry(frame, textvariable=cost)
cost_entry.grid(column=1, row=1)

is_both_sided = tk.BooleanVar(root, True)
ttk.Checkbutton(frame, text="Double sided", variable=is_both_sided).grid(column=0, columnspan=2, row=2)

ttk.Separator(frame, orient='horizontal').grid(column=0, columnspan=2, row=3, ipady=10)
ttk.Label(frame, text="Calculate path:").grid(column=0, columnspan=2, row=4)
ttk.Label(frame, text="From Node #").grid(column=0, row=5)
path_from_index = tk.IntVar(root, 1)
ttk.Entry(frame, textvariable=path_from_index).grid(column=1, row=5)
ttk.Label(frame, text="To Node #").grid(column=0, row=6)
path_to_index = tk.IntVar(root, 1)
ttk.Entry(frame, textvariable=path_to_index).grid(column=1, row=6)
methods_set = set(methods.keys())
selected_method = tk.StringVar(root, list(methods_set)[0])
ttk.Label(frame, text="Method:").grid(column=0, row=7)
ttk.OptionMenu(frame, selected_method, list(methods_set)[0], *methods_set).grid(column=1, row=7)
calc_path_button = ttk.Button(frame, text="Find path", command=on_calculate_path_button)
calc_path_button.grid(column=0, columnspan=2, row=8)
path_cost_label = ttk.Label(frame, text='')
path_cost_label.grid(column=0, columnspan=2, row=9)

root.mainloop()

Полученная программа с графических интерфейсом позволяет:
- Создавать ориентированный взвешенный граф
- Просчитывать и отображать путь между любыми двумя вершинами графа, с применением выбранного алгоритма поиска

Демонстрационный скриншот:

![Скриншот](https://i.ibb.co/m900NLN/image.png)

In [11]:
# Подсчёт сложности в зависимости от "размеров" графа
# Графы для этого генерируются, все вершины соединяются друг с другом
import time

def generate_simple_graph(nodes_count):
    nodes = [Node(i) for i in range(nodes_count)]
    for n in nodes:
        for n2 in nodes:
            if n != n2:
                n.connect_to(n2, 1)
    return nodes

def benchmark_pathfinding(nodes_count):
    def cell(s):
        width = 15
        return " " + s + ' ' * (width - len(s)) + " |"
    def print_cell(s):
        print(cell(s), end='')

    methods = [
        ("Floyd-Warshall", FloydWarshall.calc_path),
        ("Dijkstra", Dijkstra.calc_path),
        ("Bellman-Ford", BellmanFord.calc_path),
        ("Johnson", Johnson.calc_path),
        ("Levit", Levit.calc_path),
        ("Yen", Yen.calc_path)
    ]

    print_cell(f"Nodes Count")
    for name, func in methods:
        print_cell(name)
    print()
    print('-' * 18 * 7)

    for i in range(nodes_count[0], nodes_count[1], nodes_count[2]):
        nodes = generate_simple_graph(i)
        print_cell(f"{i}")
        for name, func in methods:
            start_time = time.time()
            func(nodes[0], nodes[-1], nodes)
            total_time = round((time.time() - start_time)*1000)
            print_cell(f"{total_time} ms")
        print()

benchmark_pathfinding((10, 100, 10))

 Nodes Count     | Floyd-Warshall  | Dijkstra        | Bellman-Ford    | Johnson         | Levit           | Yen             |
------------------------------------------------------------------------------------------------------------------------------
 10              | 0 ms            | 0 ms            | 0 ms            | 1 ms            | 0 ms            | 0 ms            |
 20              | 3 ms            | 0 ms            | 3 ms            | 4 ms            | 0 ms            | 1 ms            |
 30              | 9 ms            | 1 ms            | 10 ms           | 15 ms           | 1 ms            | 1 ms            |
 40              | 22 ms           | 2 ms            | 32 ms           | 33 ms           | 0 ms            | 2 ms            |
 50              | 41 ms           | 4 ms            | 88 ms           | 79 ms           | 0 ms            | 3 ms            |
 60              | 99 ms           | 12 ms           | 127 ms          | 143 ms          | 1 ms            | 8 

### Вывод

Мы реализовали 6 алгоритмов для поиска кратчайшего пути между вершинами ориентированного взвешенного графа, реализовали задание графа через файл с таблицей смежности или через графический интерфейс, и вывод пути в графический интерфейс. Кроме того, приведена таблица соотношения количества вершин в графе со временем работы алгоритмов для поиска пути по нему.